# 회귀 (Regression)

1. 목적: 연속적인 값(예: 가격, 온도, 시간 등)을 예측하는 것이 목적입니다.

2. 출력 유형: 실수 값(연속 값)을 출력합니다. 예를 들어, 집의 가격이나 자동차의 연비 등이 회귀 문제의 예시입니다.

3. 모델 예시: 선형 회귀, 다중 회귀, 릿지(Ridge) 및 라쏘(Lasso) 회귀, 다항 회귀, 서포트 벡터 머신 등.

4. 평가 방법: 평균 제곱 오차(Mean Squared Error, MSE), 평균 절대 오차(Mean Absolute Error, MAE), 결정 계수(R²) 등으로 평가합니다